In [1]:
%load_ext dotenv
%dotenv secrets/secrets.env

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic.chat_models import ChatAnthropic
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from typing import Literal
from neo4j import GraphDatabase

os.environ["LANGCHAIN_PROJECT"] = "hospital-system"

In [3]:
NODES = ["Hospital", "Payer", "Physician", "Patient", "Visit", "Reviews"]

def _set_uniqueness_constraints(tx, node):
    query = f"""CREATE CONSTRAINT IF NOT EXISTS FOR (n:{node})
        REQUIRE n.id IS UNIQUE;"""
    _ = tx.run(query, {})
    

driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)
with driver.session(database="neo4j") as session:
    for node in NODES:
        session.execute_write(_set_uniqueness_constraints, node)
    

In [4]:
NODES = ["Hospital", "Payer", "Physician", "Patient", "Visit", "Reviews"]

def _set_uniqueness_constraints(tx, node):
    query = f"""CREATE CONSTRAINT IF NOT EXISTS FOR (n:{node})
        REQUIRE n.id IS UNIQUE;"""
    _ = tx.run(query, {})
    

driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)
with driver.session(database="neo4j") as session:
    for node in NODES:
        session.execute_write(_set_uniqueness_constraints, node)
    

In [5]:
HOSPITALS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/hospitals.csv"

with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{HOSPITALS}' AS hospitals
    MERGE (h:Hospital {{id: toInteger(hospitals.hospital_id),
                            name: hospitals.hospital_name,
                            state_name: hospitals.hospital_state}});
    """
    _ = session.run(query, {})

In [6]:
PAYERS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/payers.csv"

with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PAYERS}' AS payers
    MERGE (p:Payer {{id: toInteger(payers.payer_id),
    name: payers.payer_name}});
    """
    _ = session.run(query, {})

In [7]:
PHYSICIANS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/physicians.csv"

with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PHYSICIANS}' AS physicians
    MERGE (p:Physician {{id: toInteger(physicians.physician_id),
                        name: physicians.physician_name,
                        dob: physicians.physician_dob,
                        grad_year: physicians.physician_grad_year,
                        school: physicians.medical_school,
                        salary: toFloat(physicians.salary)
                        }});
    """
    _ = session.run(query, {})

In [8]:
VISITS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/visits.csv"

with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MERGE (v:Visit {{id: toInteger(visits.visit_id),
                        room_number: toInteger(visits.room_number),
                        admission_type: visits.admission_type,
                        admission_date: visits.date_of_admission,
                        test_results: visits.test_results,
                        status: visits.visit_status
    }})
        ON CREATE SET v.chief_complaint = visits.chief_complaint
        ON MATCH SET v.chief_complaint = visits.chief_complaint
        ON CREATE SET v.treatment_description = visits.treatment_description
        ON MATCH SET v.treatment_description = visits.treatment_description
        ON CREATE SET v.diagnosis = visits.primary_diagnosis
        ON MATCH SET v.diagnosis = visits.primary_diagnosis
        ON CREATE SET v.discharge_date = visits.discharge_date
        ON MATCH SET v.discharge_date = visits.discharge_date
        """
    _ = session.run(query, {})

In [9]:
PATIENTS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/patients.csv"

with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS
    FROM '{PATIENTS}' AS patients
    MERGE (p:Patient {{id: toInteger(patients.patient_id),
                    name: patients.patient_name,
                    sex: patients.patient_sex,
                    dob: patients.patient_dob,
                    blood_type: patients.patient_blood_type
                    }});
    """
    _ = session.run(query, {})

In [10]:
REVIEWS = "https://raw.githubusercontent.com/sakunaharinda/ragatouille/main/data/reviews.csv"

with driver.session(database="neo4j") as session:
        query = f"""
        LOAD CSV WITH HEADERS
        FROM '{REVIEWS}' AS reviews
        MERGE (r:Review {{id: toInteger(reviews.review_id),
                         text: reviews.review,
                         patient_name: reviews.patient_name,
                         physician_name: reviews.physician_name,
                         hospital_name: reviews.hospital_name
                        }});
        """
        _ = session.run(query, {})

In [11]:
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MATCH (source: `Visit` {{ `id`: toInteger(trim(visits.visit_id)) }})
    MATCH (target: `Hospital` {{ `id`: toInteger(trim(visits.hospital_id))}})
    MERGE (source)-[r: `AT`]->(target)
    """
    _ = session.run(query, {})
    
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MATCH (source: `Patient` {{ `id`: toInteger(visits.patient_id) }})
    MATCH (target: `Visit` {{ `id`: toInteger(visits.visit_id) }})
    MERGE (source)-[: `HAS`]->(target)
    """
    _ = session.run(query, {})
    
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MATCH (source: `Visit` {{ `id`: toInteger(visits.visit_id) }})
    MATCH (target: `Payer` {{ `id`: toInteger(visits.payer_id) }})
    MERGE (source)-[covered_by: `COVERED_BY`]->(target)
    ON CREATE SET
        covered_by.service_date = visits.discharge_date,
        covered_by.billing_amount = toFloat(visits.billing_amount) 
    """
    _ = session.run(query, {})
    
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MATCH (source: `Hospital` {{ `id`: toInteger(visits.hospital_id) }})
    MATCH (target: `Physician` {{ `id`: toInteger(visits.physician_id) }})
    MERGE (source)-[: `EMPLOYS`]->(target)
    """
    _ = session.run(query, {})
    
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{VISITS}' AS visits
    MATCH (source: `Physician` {{ `id`: toInteger(visits.physician_id) }})
    MATCH (target: `Visit` {{ `id`: toInteger(visits.visit_id) }})
    MERGE (source)-[: `TREATS`]->(target)
    """
    _ = session.run(query, {})
    
with driver.session(database="neo4j") as session:
    query = f"""
    LOAD CSV WITH HEADERS FROM '{REVIEWS}' AS reviews
    MATCH (source: `Visit` {{ `id`: toInteger(reviews.visit_id) }})
    MATCH (target: `Review` {{ `id`: toInteger(reviews.review_id) }})
    MERGE (source)-[: `WRITES`]->(target)
    """
    _ = session.run(query, {})

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD'),
    url=os.getenv('NEO4J_URI'),
    node_label="Review",
    index_name="review_vector_index",
    text_node_properties=[
        "review", "physician_name", "hospital_name", "patient_name"
    ],
    embedding_node_property="embedding"
)

review_retriever = neo4j_vector_index.as_retriever()